BeautifulSoup: 웹 스크래핑의 일종.

In [8]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import ssl

로그인이 필요한 페이지 접속
* https://www.fun-coding.org/crawl_advance2.html 참고함
* 크롬 개발자모드는--> 상단 바 오른쪽에 점세개 누르면 [도구 더보기]->[개발자 도구]
* 전반적인 코드가 보인다.

In [32]:
login_url='http://www.solarcube.co.kr/SrMain/SM010.aspx'
user = 'idnstorymrps'
password = 'goodbye7669!'

In [33]:
session = requests.session()

params = dict()
params['uid'] = user
params['upwd'] = password

In [36]:
res = session.post(login_url, data = params)
#--- OK가 안되면 에러코드를 프린트한다고 함
res.raise_for_status(200)

TypeError: raise_for_status() takes 1 positional argument but 2 were given

In [35]:
res

<Response [200]>

Set-Cookie로 PHPSESSID라는 세션 ID 값이 넘어온다고 함.

In [17]:
print(res.headers)

{'Cache-Control': 'private', 'Content-Type': 'text/html; charset=utf-8', 'Server': 'Microsoft-IIS/10.0', 'X-AspNet-Version': '4.0.30319', 'X-Powered-By': 'ASP.NET', 'Date': 'Thu, 03 Dec 2020 03:20:27 GMT', 'Content-Length': '16945'}


cookie로 세선을 로그인 상태를 관리하는 상태를 확인하려면

In [18]:
print(session.cookies.get_dict())

{'ASP.NET_SessionId': '5ttabofjhqwheo2yfudu0ocn'}


여기서부터는 로그인이 이미 된 상태라고 함.

In [28]:
url_page = 'http://www.solarcube.co.kr/SrStMng/SSM010.aspx'
res = session.get(url_page)

res.raise_for_status()

In [31]:
res.text

'<script language=\'javascript\'>location.replace("../");</script>'

In [30]:
soup = BeautifulSoup(res, 'html.parser')

TypeError: object of type 'Response' has no len()

전체 내용을 다 보고 싶을 때

In [21]:
print(soup.prettify())

<script language="javascript">
 location.replace("../");
</script>


아래 포함된 태그를 보고 싶을 때

In [22]:
list(soup.children)

[<script language="javascript">location.replace("../");</script>]

바로 body를 추출해내려면

In [ ]:
soup.body

찾고있는 태그를 알고 있을 때

In [ ]:
#soup.find('p') ## 하나만 찾을 때
soup.find_all('p') ## 전부 찾을 때


In [ ]:
soup.find_all(class_='inner-text') ## class 이름으로 찾을 때

### 텍스트 안에 있는 태그만 가져올 경우

In [ ]:
for each_tag in soup.find_all('p'):
    print(each_tag.get_text())

In [ ]:
for each_tag in soup.find_all('a'):
    print(each_tag.get_text(),'->',each_tag['href'])

# 네이버 페이지에서 시도!!

In [82]:
url='https://finance.naver.com/marketindex/'


context = ssl._create_unverified_context()

page = urlopen(url, context=context)

soup = BeautifulSoup(page,'html.parser')

In [83]:
print(soup.prettify())

<script language="javascript" src="/template/head_js.nhn?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market">
</script>
<script src="/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
</script>
<script src="/js/jindo.1.5.3.element-text-patch.js" type="text/javascript">
</script>
<div id="container" style="padding-bottom:0px;">
 <script language="JavaScript" src="/js/flashObject.js?20201030171152">
 </script>
 <div class="market_include">
  <div class="market_data">
   <div class="market1">
    <div class="title">
     <h2 class="h_market1">
      <span>
       환전 고시 환율
      </span>
     </h2>
    </div>
    <!-- data -->
    <div class="data">
     <ul class="data_lst" id="exchangeList">
      <li class="on">
       <a class="head usd" href="/marketindex/exchangeDetail.nhn?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
        <h3 class="h_lst">
         <span class="blind">
          미국 USD
         </span>
        </h3>
 

In [85]:
i = 0
for each_tag in soup.find_all('span'):
    print(each_tag,i)
    i=i+1

<span>환전 고시 환율</span> 0
<span class="blind">미국 USD</span> 1
<span class="value">1,113.00</span> 2
<span class="txt_krw"><span class="blind">원</span></span> 3
<span class="blind">원</span> 4
<span class="change"> 4.00</span> 5
<span class="blind">하락</span> 6
<span class="time">2020.11.11 11:36</span> 7
<span class="source">하나은행 기준</span> 8
<span class="count">고시회차<span class="num">140</span>회</span> 9
<span class="num">140</span> 10
<span class="blind">일본 JPY(100엔)</span> 11
<span class="value">1,058.64</span> 12
<span class="txt_krw"><span class="blind">원</span></span> 13
<span class="blind">원</span> 14
<span class="change"> 0.43</span> 15
<span class="blind">하락</span> 16
<span class="time">2020.11.11 11:36</span> 17
<span class="source">하나은행 기준</span> 18
<span class="count">고시회차<span class="num">140</span>회</span> 19
<span class="num">140</span> 20
<span class="blind">유럽연합 EUR</span> 21
<span class="value">1,315.84</span> 22
<span class="txt_krw"><span class="blind">원</span></span> 23


In [86]:
import pandas as pd

In [91]:
save_blind = []
for each_tag in soup.find_all('span','blind'):
    save_blind.append(each_tag.get_text())
    
save_value = []
for each_tag in soup.find_all('span','value'):
    save_value.append(each_tag.get_text())
    
    
save_change = []
for each_tag in soup.find_all('span','change'):
    save_change.append(each_tag.get_text())
    
save_txt_krw = []
for each_tag in soup.find_all('span','txt_krw'):
    save_txt_krw.append(each_tag.get_text())

In [92]:
len(save_change)

12

### 그래프 페이지

In [104]:
for ii in range(0,len(save_value)-1):
    if ii<7:
        print(save_blind[ii*3]+': '+save_value[ii]+' '+save_blind[ii*3+1]+' ('+save_change[ii]+save_blind[ii*3+2]+')')
    elif ii==7:
        print(save_blind[ii*3]+': '+save_value[ii]+' ('+save_change[ii]+save_blind[ii*3+1]+')')
    else:
        print(save_blind[ii*3-1]+': '+save_value[ii]+' '+save_blind[ii*3]+' ('+save_change[ii]+save_blind[ii*3+1]+')')


미국 USD: 1,113.00 원 ( 4.00하락)
일본 JPY(100엔): 1,058.64 원 ( 0.43하락)
유럽연합 EUR: 1,315.84 원 ( 0.38하락)
중국 CNY: 168.83 원 ( 0.25하락)
일본 엔/달러: 105.3800 엔 (0.0900상승)
달러/유로: 1.1812 달러 ( 0.0054하락)
달러/영국파운드: 1.3256 달러 (0.0094상승)
달러인덱스: 92.7300 (0.0200상승)
WTI: 41.36 달러 (1.07상승)
휘발유: 1318.08 원 ( 0.06하락)
국제 금: 1875.4 달러 (22.20상승)


In [108]:
soup.find_all('div','calculator')

[<div class="calculator">
 <div class="inner">
 <div class="input">
 <div class="selectbox-noscript" id="sInput">
 <select class="selectbox-source" id="select_from">
 <option label="1" value="1">대한민국 원 KRW</option>
 <option class="selectbox-default" label="1" selected="selected" value="1113.0"> 미국 달러 USD</option>
 <option label="1" value="1113.0"> 미국 달러 USD</option>
 <option label="1" value="1315.84"> 유럽연합 유로 EUR</option>
 <option label="100" value="10.586400000000001"> 일본 엔 JPY</option>
 <option label="1" value="168.83"> 중국 위안 CNY</option>
 <option label="1" value="143.54"> 홍콩 달러 HKD</option>
 <option label="1" value="38.99"> 대만 달러 TWD</option>
 <option label="1" value="1476.84"> 영국 파운드 GBP</option>
 <option label="1" value="2890.83"> 오만 리알 OMR</option>
 <option label="1" value="854.94"> 캐나다 달러 CAD</option>
 <option label="1" value="1217.32"> 스위스 프랑 CHF</option>
 <option label="1" value="129.26"> 스웨덴 크로나 SEK</option>
 <option label="1" value="811.54"> 호주 달러 AUD</option>
 <option label

In [ ]:
for each_tag in soup.find_all('span','blind'):
    save_blind.append(each_tag.get_text())
    

# 해줌 페이지에서 시도
* IncompleteRead가 발생해서.. urlopen 대신 requests 사용해봄 (https://stackoverflow.com/questions/51226635/http-client-incompleteread-error-in-python3 참고함) 

In [5]:
import requests

In [13]:
url='https://www.haezoom.com/mini/'

page = requests.get(url)

soup = BeautifulSoup(page.text,'html.parser')

In [14]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 8]><html lang="kr" class="ie8"><![endif]-->
<!--[if IE 9]><html lang="kr" class="ie9"><![endif]-->
<html lang="kr">
 <head>
  <title>
   국내 1위 태양광 업체 l 해줌
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible" id="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="_JavTyxA17VT5fv-nNeyWAie0yTinfhgyey8PhMea0w" name="google-site-verification"/>
  <meta content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0" name="viewport"/>
  <meta content="haezoom" property="og:site_name"/>
  <meta content="website" property="og:type"/>
  <meta content="해줌 l 미니태양광" property="og:title"/>
  <meta content="해줌 프리미엄 미니 태양광 서비스 제공&lt;br&gt;어디에나 편리하게 설치하고 전기료를 절감하세요. 더 따뜻하고, 더 쾌적한 우리집 발전소" property="og:description"/>
  <meta content="/media/og_image/solarmini_img_1" property="og:image"/>
  <link href="/s/assets/img/haezoom-favicon.ico" rel="shor

In [19]:
for each_tag in soup.find_all('div'):
    #print(each_tag)
    print(each_tag.get_text())
    #for each_class in each_tag.find_all(class_='value'):
    #    print(each_class.get_text())

미니/베란다 태양광 설치해줌 미니/베란다 태양광우리집 전기료를 디자인하는 따뜻한 발전소지금 문의하기
우리집 전기요금은 얼마나 절감될까요?한달 평균 전기 사용량kWh한달 평균 사용 전기량을 입력해주세요계산하기절감 전 전기료절감액절감 후 전기료※ 태양광 400W, 일 2.75시간 발전 기준
우리집 전기요금은 얼마나 절감될까요?
한달 평균 전기 사용량kWh한달 평균 사용 전기량을 입력해주세요계산하기
한달 평균 전기 사용량kWh한달 평균 사용 전기량을 입력해주세요계산하기
한달 평균 전기 사용량kWh한달 평균 사용 전기량을 입력해주세요계산하기
한달 평균 전기 사용량kWh한달 평균 사용 전기량을 입력해주세요
계산하기
절감 전 전기료절감액절감 후 전기료
절감 전 전기료절감액절감 후 전기료
절감 전 전기료절감액절감 후 전기료
절감 전 전기료절감액절감 후 전기료
절감 전 전기료절감액절감 후 전기료
절감 전 전기료
절감 전 전기료

절감액
절감액


절감 후 전기료
절감 후 전기료

※ 태양광 400W, 일 2.75시간 발전 기준
※ 태양광 400W, 일 2.75시간 발전 기준
※ 태양광 400W, 일 2.75시간 발전 기준
태양광 설치부터 IT기술까지,태양광을 이끌어가는 리더고객을 최우선하는 최고의 서비스가정용 태양광 1위아파트 태양광 1위국내 유일 직영 A/S팀사후관리 전문팀이직접 관리해 신뢰도 상승국내최초 벽면 태양광발전량 예측 기술 보유체계적인 서비스 제공
고객을 최우선하는 최고의 서비스가정용 태양광 1위아파트 태양광 1위국내 유일 직영 A/S팀사후관리 전문팀이직접 관리해 신뢰도 상승국내최초 벽면 태양광발전량 예측 기술 보유체계적인 서비스 제공
고객을 최우선하는 최고의 서비스가정용 태양광 1위아파트 태양광 1위국내 유일 직영 A/S팀사후관리 전문팀이직접 관리해 신뢰도 상승국내최초 벽면 태양광발전량 예측 기술 보유체계적인 서비스 제공
고객을 최우선하는 최고의 서비스가정용 태양광 1위아파트 태양광 1위
고객을 최우선하는 최고의 서비스가정용 태양광 1위아파트 태양광 1위
국내

# 기상청 홈페이지를 뜯어보자

In [22]:
url='https://www.weather.go.kr/weather/observation/currentweather.jsp'

page = requests.get(url)

soup = BeautifulSoup(page.text,'html.parser')

In [23]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <title>
   도시별 현재날씨 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청
  </title>
  <link href="https://www.kma.go.kr/iphone-shortcut.png" rel="shortcut icon"/>
  <link href="https://www.kma.go.kr/iphone-shortcut.png" rel="apple-touch-icon-precomposed"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/share/css/base.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <link href="/share/css/common.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <link href="/share/css/weather-layout.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <link href="/share/css/component.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <link href="/share/css/add_2018.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <link href="/share/css/weather-obs.css?ver=20200928" rel="stylesheet" type="text/css"/>
  <script src="/share/js/jquery-1.7.1.min.js">
  </script>
 